In [1]:
include("Utils.jl")
include("Spread.jl")
using .SpreadUtils
using .PolySpreads

In [2]:
poly_spread_N(8, (3,3), 1)

Dict{Any, Any} with 6 entries:
  (2, 4) => 1//6
  (3, 3) => 2//9
  (2, 2) => 1//36
  (3, 4) => 5//18
  (2, 3) => 5//18
  (4, 4) => 1//36

In [3]:
function free_spread_tup(tup, location, N)
    res = Dict()
    
    tup = [tup...]
    # coeff = pair[2]

    i = tup[location]
    
    
    if i == 1
        spread_tup_right = deepcopy(tup)
        spread_tup_right[location] = 2
        # spread_tup_right = SpreadUtils.ordered(spread_tup_right...)
        res = Dict(Tuple(tup)=>  3//4,  Tuple(spread_tup_right) =>  1//4)
    elseif i == N
        spread_tup_left = deepcopy(tup)
        spread_tup_left[location] = N-1
        # spread_tup_left = SpreadUtils.ordered(spread_tup_left...)
        res = Dict(Tuple(tup)=>  3//4,  Tuple(spread_tup_left) =>  1//4)
    else
        spread_tup_left = deepcopy(tup)
        spread_tup_left[location] = i-1
        spread_tup_right = deepcopy(tup)
        spread_tup_right[location] = i+1

        # spread_tup_left = SpreadUtils.ordered(spread_tup_left...)
        # spread_tup_right = SpreadUtils.ordered(spread_tup_right...)
        res = Dict(Tuple(tup)=>  1//2,  Tuple(spread_tup_left) =>  1//4, Tuple(spread_tup_right) =>  1//4)
    end

    return res
end


function free_spread(diction, N)
    res = Dict()
    for (key, value) in diction
        spread1 = free_spread_tup(key, 1, N)
        for (key2, value2) in spread1
            spread2 = free_spread_tup(key2, 2, N)
            for (key3, value3) in spread2
                key3 = SpreadUtils.ordered(key3...)
                new_dict = Dict(key3 => value3*value2)
                mergewith!(+, res, new_dict)
                # push!(res, ) 
            end
        end
    end
    return res
end

free_spread (generic function with 1 method)

In [4]:
free_spread_tup((1,1), 1, 3)

Dict{Tuple{Int64, Int64}, Rational{Int64}} with 2 entries:
  (1, 1) => 3//4
  (2, 1) => 1//4

In [11]:
N = 16
d = (1,1)

spreaded_dict = poly_spread_N(N, d,1)
free_spread_dict = free_spread(Dict(d => 1//1), N)

interact_dict = mergewith(-, spreaded_dict, free_spread_dict)
filter(pair -> pair[2] != 0, interact_dict)

Dict{Any, Any} with 3 entries:
  (1, 2) => 5//72
  (1, 1) => -5//144
  (2, 2) => -5//144

In [12]:
free_spread_dict

Dict{Any, Any} with 3 entries:
  (1, 2) => 3//8
  (1, 1) => 9//16
  (2, 2) => 1//16

In [7]:
sum(free_spread_dict) do pair
    pair[2]
end

1//1

In [8]:
spreaded_dict

Dict{Any, Any} with 8 entries:
  (2, 4) => 1//8
  (1, 2) => 1//16
  (3, 3) => 5//48
  (1, 3) => 1//8
  (2, 2) => 5//48
  (3, 4) => 1//16
  (1, 4) => 1//16
  (2, 3) => 17//48